## Exercises I

Let's review the steps we take at the beginning of each new module.

1. Create a new repository named regression-exercises in your GitHub; all of your Regression work will be housed here.

2. Clone this repository within your local codeup-data-science directory.

3. Create a .gitignore and make sure your list of 'files to ignore' includes your env.py
file.

4. Ceate a README.md file that outlines the contents and purpose of your repository.

5. Add,commit,andpushthesetwofiles.

6. Now you can add your env.py file to this repository to access the Codeup database server.

7. For these exercises, you will create wrangle.ipynb and wrangle.py files to hold necessary functions.

8. Asalways,add,commit,andpushyourworkoften.


## Exercises II ( Acquire & Prepare - Wrangle )

Let's set up an example scenario as perspective for our regression exercises using the Zillow dataset.

As a Codeup data science graduate, you want to show off your skills to the Zillow data science team in hopes of getting an interview for a position you saw pop up on LinkedIn. You thought it might look impressive to build an end-to-end project in which you use some of their Kaggle data to predict property values using some of their available features; who knows, you might even do some feature engineering to blow them away. **Your goal is to predict the values of single unit properties using the observations from 2017**.

In these exercises, you will complete the first step toward the above goal: acquire and prepare the necessary Zillow data from the zillow database in the Codeup database server.

1. Acquire bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet, taxvaluedollarcnt , yearbuilt , taxamount , and fips from the zillow database for all 'Single Family Residential' properties.

2. Using your acquired Zillow data,walk through the summarization and cleaning steps in your wrangle.ipynb file like we did above. You may handle the missing values however you feel is appropriate and meaningful; remember to document your process and decisions using markdown and code commenting where helpful.

3. Write a function to split your data into train,validate,and test.

4. Store all of the necessary functions to automate your process from acquiring the data to returning a cleaned dataframe with no missing values in your wrangle.py file. Name your final function wrangle_zillow .

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import env
import wrangle as w

import os

> Q1) Acquire bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet, taxvaluedollarcnt , yearbuilt , taxamount , and fips from the zillow database for all 'Single Family Residential' properties.

In [2]:
url =env.get_db_url('zillow')

In [3]:
query='''
            select 
            bedroomcnt,
            bathroomcnt,
            calculatedfinishedsquarefeet,
            taxvaluedollarcnt, 
            yearbuilt, 
            taxamount, 
            fips
            from properties_2017
            where propertylandusetypeid='261'
            limit 100
            
            '''

<div class='alert alert-block alert-success'>
NOTE:
Above I looked at only 100 rows of data.

Because I know Zillow is a large dataset, I will create a function that runs the same query with ALL the data, but I will make sure to cache these results for future use!



</div class='alert alert-block alert-success'>

In [4]:
zillow_sample=pd.read_sql(query,url)
zillow_sample

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0
...,...,...,...,...,...,...,...
95,3.0,1.5,1698.0,286606.0,1979.0,2987.36,6111.0
96,3.0,1.0,1058.0,185908.0,1959.0,2764.58,6111.0
97,3.0,2.0,1688.0,285106.0,1958.0,3659.32,6111.0
98,3.0,2.0,1066.0,54684.0,1958.0,1260.60,6111.0


In [5]:
def get_zillow_data():
    filename = "zillow.csv"

    if os.path.isfile(filename):

        return pd.read_csv(filename, index_col=0)
    else:
        # Create the url
        url = env.get_db_url('zillow')
        
        sql_query = '''
            SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, 
            taxvaluedollarcnt, yearbuilt, taxamount, fips
            FROM properties_2017
            WHERE propertylandusetypeid = 261'''

        # Read the SQL query into a dataframe
        df = pd.read_sql(sql_query, url)

        # Write that dataframe to disk for later. Called "caching" the data for later.
        df.to_csv(filename)

        # Return the dataframe to the calling code
        return df

In [6]:
df=get_zillow_data()
df.head(4)

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0


> 2)Using your acquired Zillow data,walk through the summarization and cleaning steps in your wrangle.ipynb file like we did above. You may handle the missing values however you feel is appropriate and meaningful; remember to document your process and decisions using markdown and code commenting where helpful.

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2152863 entries, 0 to 2152862
Data columns (total 7 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   bedroomcnt                    float64
 1   bathroomcnt                   float64
 2   calculatedfinishedsquarefeet  float64
 3   taxvaluedollarcnt             float64
 4   yearbuilt                     float64
 5   taxamount                     float64
 6   fips                          float64
dtypes: float64(7)
memory usage: 131.4 MB


In [8]:
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 2152863 entries, 0 to 2152862
Data columns (total 7 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   bedroomcnt                    2152852 non-null  float64
 1   bathroomcnt                   2152852 non-null  float64
 2   calculatedfinishedsquarefeet  2144379 non-null  float64
 3   taxvaluedollarcnt             2152370 non-null  float64
 4   yearbuilt                     2143526 non-null  float64
 5   taxamount                     2148421 non-null  float64
 6   fips                          2152863 non-null  float64
dtypes: float64(7)
memory usage: 131.4 MB


In [9]:
#let's make this more readable
df = df.rename(columns={'bedroomcnt':'bedrooms',
                  'bathroomcnt':'bathrooms',
                  'calculatedfinishedsquarefeet':'area',
                  'taxvaluedollarcnt':'property_value',
                  'fips':'county'})

In [10]:
df.head(3)

,bedrooms,bathrooms,area,property_value,yearbuilt,taxamount,county
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0


In [11]:
df.shape

(2152863, 7)

In [12]:
# Find the total number of Null values in each column of our DataFrame.
df.isnull().sum()

bedrooms            11
bathrooms           11
area              8484
property_value     493
yearbuilt         9337
taxamount         4442
county               0
dtype: int64

In [13]:
# another way
#let's turn our column values into a list
col = df.columns.to_list()
col

['bedrooms',
 'bathrooms',
 'area',
 'property_value',
 'yearbuilt',
 'taxamount',
 'county']

In [14]:
#are we dealing with any nulls?
num_missing = df[col].isna().sum()
num_missing

bedrooms            11
bathrooms           11
area              8484
property_value     493
yearbuilt         9337
taxamount         4442
county               0
dtype: int64

In [15]:
#what percent of my entire dataset is this?
pct_missing = num_missing / df.shape[0]
pct_missing

bedrooms          0.000005
bathrooms         0.000005
area              0.003941
property_value    0.000229
yearbuilt         0.004337
taxamount         0.002063
county            0.000000
dtype: float64

<div class='alert alert-block alert-success'>
NOTE:


* bedrooms and bathrooms is less than .001% null

* area is 0.4% null

* tax_value is 0.02% null

* year_built is 0.4% null

* taxamount is 0.2% null

Therefore, dropping our nulls would be our best bet in this case.
</div class='alert alert-block alert-success'>

In [16]:
#let's drop those null values
df = df.dropna()

In [17]:
#confirm our changes were made
df.isnull().sum()

bedrooms          0
bathrooms         0
area              0
property_value    0
yearbuilt         0
taxamount         0
county            0
dtype: int64

In [18]:
df.dtypes

bedrooms          float64
bathrooms         float64
area              float64
property_value    float64
yearbuilt         float64
taxamount         float64
county            float64
dtype: object

In [19]:
df.head()

,bedrooms,bathrooms,area,property_value,yearbuilt,taxamount,county
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0
6,3.0,4.0,1620.0,847770.0,2011.0,10244.94,6037.0
7,3.0,2.0,2077.0,646760.0,1926.0,7924.68,6037.0
11,0.0,0.0,1200.0,5328.0,1972.0,91.60,6037.0
14,0.0,0.0,171.0,6920.0,1973.0,255.17,6037.0


In [20]:
#confirm whole numbers
for col in df.columns:
    print(col)
    print(df[col].value_counts().sort_index())
    print()

bedrooms
bedrooms
0.0       4397
1.0      22895
2.0     334221
3.0     962944
4.0     633608
5.0     150671
6.0      25117
7.0       4792
8.0       1103
9.0        290
10.0       118
11.0        34
12.0        12
13.0        15
14.0         7
15.0         5
16.0         2
18.0         3
25.0         1
Name: count, dtype: int64

bathrooms
bathrooms
0.00       4274
0.50         16
1.00     412582
1.50      31157
1.75          3
2.00     942463
2.50     142827
3.00     422398
3.50      28464
4.00      82039
4.50      19474
5.00      28306
5.50       6201
6.00      10717
6.50       1330
7.00       4381
7.50        382
8.00       1681
8.50        108
9.00        707
9.50         50
10.00       322
10.50        14
11.00       145
11.50         3
12.00        73
12.50         3
13.00        39
14.00        25
14.50         1
15.00        17
16.00        12
17.00         4
18.00         8
19.00         1
19.50         1
20.00         6
32.00         1
Name: count, dtype: int64

area
area
1.0  

In [21]:
#possibly an easier way to do this... create a lamda function
#let's see if there are any fractions in bathrooms
has_non_zero_decimal = df['bathrooms'].apply(lambda x: x % 1 != 0)
has_non_zero_decimal.any()

True

In [22]:
#how many?
has_non_zero_decimal.sum()

230034

In [23]:
#what about bedrooms?
has_non_zero_decimal = df['bedrooms'].apply(lambda x: x % 1 != 0)
has_non_zero_decimal.sum()

0

In [24]:
#put it all together for all the columns 
##so we can make sure those floats should be integers

#include these columns
columns_to_check = ['bedrooms', 'bathrooms', 'area', 'property_value', 'yearbuilt', 'taxamount', 'county']

# create a dictionary to store the count of non-zero decimal places for each column
count_of_non_zero_decimal = {}

# iterate through columns and check for non-zero decimal places
for column in columns_to_check:
    has_non_zero_decimal = (df[column] % 1 != 0).sum()
    
    # store the count in the dictionary
    count_of_non_zero_decimal[column] = has_non_zero_decimal

# display the count of non-zero decimal places for each column
print("Count of Non-Zero Decimal Places in Each Column:")
for column, count in count_of_non_zero_decimal.items():
    print(f"{column}: {count}")

Count of Non-Zero Decimal Places in Each Column:
bedrooms: 0
bathrooms: 230034
area: 0
property_value: 0
yearbuilt: 0
taxamount: 2111620
county: 0


<div class='alert alert-block alert-success'>
NOTE:
It looks like several of these features can be changed to integers

* bedrooms (can't have a fraction of a bedroom)

* area (all appear to be whole numbers)

* property_value (all appear to be whole numbers)

* yearbuilt (can't have a fraction of a year)

* county (can't have a fraction of a county)
</div class='alert alert-block alert-success'>

In [25]:
#columns we want to be integers
make_ints = ['bedrooms','area','property_value','yearbuilt','county']

#look through them to change the datatype
for col in make_ints:
    df[col] = df[col].astype(int)

In [26]:
df.dtypes

bedrooms            int64
bathrooms         float64
area                int64
property_value      int64
yearbuilt           int64
taxamount         float64
county              int64
dtype: object

In [27]:
#take a look
df.head()

,bedrooms,bathrooms,area,property_value,yearbuilt,taxamount,county
4,4,2.0,3633,296425,2005,6941.39,6037
6,3,4.0,1620,847770,2011,10244.94,6037
7,3,2.0,2077,646760,1926,7924.68,6037
11,0,0.0,1200,5328,1972,91.60,6037
14,0,0.0,171,6920,1973,255.17,6037


<div class='alert alert-block alert-success'>
NOTE:
Fips is another feature we want to investigate

* 6037: LA
* 6059: Orange County
* 6111: Ventura County
</div class='alert alert-block alert-success'>

In [28]:
df.county.value_counts()

county
6037    1425207
6059     552057
6111     162971
Name: count, dtype: int64

In [29]:
#rename those columns
df.county = df.county.map({6037:'LA',6059:'Orange',6111:'Ventura'})

In [30]:
df.sample(8)

,bedrooms,bathrooms,area,property_value,yearbuilt,taxamount,county
911895,2,2.0,1209,636853,1957,7491.46,LA
342499,2,1.5,1842,455635,1955,5340.56,Orange
1346722,4,2.0,1471,455632,1962,5270.74,Ventura
1305202,3,3.0,2888,778000,1981,9533.11,LA
271739,4,2.0,1714,155328,1990,2764.31,LA
1475853,4,2.5,1954,287212,1965,3460.08,Ventura
849636,4,3.0,3042,938542,1997,9778.20,Orange
754931,3,1.0,981,292288,1947,3608.84,LA


> 4. Store all of the necessary functions to automate your process from acquiring the data to returning a cleaned dataframe with no missing values in your wrangle.py file. Name your final function wrangle_zillow .

In [31]:
def prep_zillow(df):
    '''
    This function takes in a dataframe
    renames the columns and drops nulls values
    Additionally it changes datatypes for appropriate columns
    and renames fips to actual county names.
    Then returns a cleaned dataframe
    '''
    df = df.rename(columns = {'bedroomcnt':'bedrooms',
                     'bathroomcnt':'bathrooms',
                     'calculatedfinishedsquarefeet':'area',
                     'taxvaluedollarcnt':'property_value',
                     'fips':'county'})
    
    df = df.dropna()
    
    make_ints = ['bedrooms','area','property_value','yearbuilt']

    for col in make_ints:
        df[col] = df[col].astype(int)
        
    df.county = df.county.map({6037:'LA',6059:'Orange',6111:'Ventura'})
    
    return df


In [32]:
df_fresh=get_zillow_data()
df_fresh.sample(4)

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
1319594,3.0,3.0,1487.0,223817.0,1937.0,2789.92,6037.0
1658182,4.0,2.0,1635.0,386716.0,1965.0,4083.20,6111.0
1322595,4.0,2.5,2334.0,721842.0,1974.0,8165.76,6059.0
891113,4.0,3.0,2042.0,437938.0,1957.0,5422.40,6059.0


In [33]:
df_fresh=prep_zillow(df_fresh)
df_fresh.sample(4)

,bedrooms,bathrooms,area,property_value,yearbuilt,taxamount,county
717547,4,3.0,2690,346263,1984,4337.13,LA
309442,3,5.0,2637,617409,1957,7306.48,Orange
661891,3,1.0,1482,372000,1931,4869.93,LA
353311,3,2.0,1946,271146,1969,3296.22,Orange


In [34]:
df=w.wrangle_zillow()
df.head(3)

,bedrooms,bathrooms,area,property_value,yearbuilt,taxamount,county
4,4,2.0,3633,296425,2005,6941.39,LA
6,3,4.0,1620,847770,2011,10244.94,LA
7,3,2.0,2077,646760,1926,7924.68,LA


> 3. Write a function to split your data into train,validate,and test.


In [35]:
train, validate, test = w.split_data(df)


    train -> (1284141, 7)
    validate -> (428047, 7)
    test -> (428047, 7)


### If I wanted to have my data split within my wrangle as well, we can use this function:

In [36]:
train,validate,test = w.wrangle_zillow_split()


    train -> (1284141, 7)
    validate -> (428047, 7)
    test -> (428047, 7)
